カレントディレクトリをdarknetがある場所に移動させておく

In [ ]:
% cd '/content/drive/My Drive/darknet' 

/content/drive/My Drive/darknet


In [ ]:
#! make clean
! make 

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv

訓練用と検証用にフォルダの中の画像を自動で割り振るプログラムを初回実行時に実行する

In [ ]:
! python process.py

一度作業を中断すると許可が必要な場合がある

In [ ]:
! chmod 755 darknet

学習を実行する

In [ ]:
! ./darknet detector train /content/drive/MyDrive/darknet/task2/dataset.data /content/drive/MyDrive/darknet/task2/yolov3-voc.cfg /content/drive/MyDrive/darknet/darknet53.conv.74 -dont_show -map

ストリーミング出力は最後の 5000 行に切り捨てられました。
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 81735, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 1300 iterations) 
 Last accuracy mAP@0.50 = 98.16 %, best = 98.16 % 
 1284: 0.088447, 0.105500 avg loss, 0.001000 rate, 11.775337 seconds, 82176 images, 9.319045 hours left
Loaded: 0.000062 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.757098), count: 4, class_loss = 0.000361, iou_loss = 0.165916, total_loss = 0.166277 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normal

テストデータを読み込ませて画像サイズの変更、切り抜きを行う

In [ ]:
%cd /content/drive/MyDrive/PBL_data/work

/content/drive/MyDrive/PBL_data/work


元の画像を32の倍数にサイズ変更する

In [ ]:
import os
import cv2
import math

files = os.listdir("/content/drive/MyDrive/PBL_data/work") #画像のあるフォルダのパス名

dir_path = '/content/drive/MyDrive/PBL_data/resize_file2/padding/' #保存先のフォルダのパス名

#files.pop()
for filename in files:
    if __name__ == "__main__":
      img = cv2.imread(filename)
      kekkafile = filename.strip(".jpg")
      kekkafile = kekkafile + 'padding.jpg'

      # 縦の長さを拡張する
      diffsize = 36
      padding_half = diffsize
      color = (99, 215, 163)
      padding_img = cv2.copyMakeBorder(img, 0, padding_half, 0, 0, cv2.BORDER_CONSTANT, value=color)
      cv2.imwrite(dir_path + str(kekkafile), padding_img)

In [ ]:
print(img)

None


元画像を96×96に分割する

In [ ]:
from pathlib import Path

import cv2
import numpy as np

img = cv2.imread("/content/drive/MyDrive/PBL_data/resize_file2/padding/0808_00_Apadding.jpg")

size = (96, 96)  # 分割後の大きさ
rows = int(np.ceil(img.shape[0] / size[0]))  # 行数
cols = int(np.ceil(img.shape[1] / size[1]))  # 列数

chunks = []
for row_img in np.array_split(img, rows, axis=0):
    for chunk in np.array_split(row_img, cols, axis=1):
        chunks.append(chunk)
print(len(chunks))

# 保存する。
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)
for i, chunk in enumerate(chunks):
    save_path = output_dir / f"chunk_{i:01d}.jpg"
    cv2.imwrite(str(save_path), chunk)

1160


分割した画像をそれぞれテストする

In [ ]:
% cd '/content/drive/My Drive/darknet' 

/content/drive/My Drive/darknet


In [ ]:
import subprocess
import os

i = 0
for i in range(1160):
  subprocess.run(f"./darknet detector test /content/drive/MyDrive/darknet/task2/dataset.data /content/drive/MyDrive/darknet/task2/yolov3-voc1.cfg /content/drive/MyDrive/darknet/task2/backup/yolov3-voc_best.weights /content/drive/MyDrive/PBL_data/JPG/output/chunk_{i:d}.jpg -dont_show", shell=True)
  newpath = "/content/drive/MyDrive/working_dir2/keepimg/test" + str(i) + ".jpg"
  os.rename("/content/drive/MyDrive/darknet/predictions.jpg", newpath)

分割すると横40枚縦29枚

In [ ]:
import cv2
import numpy as np

#im1で同じ行の結合を保存 im2に目的の画像を保存 im3でくっつける画像を持つ


i = 0 # 画像名の番号

for a in range(29):
  im1 = cv2.imread('/content/drive/MyDrive/working_dir2/keepimg/test' + str(i) + '.jpg')

  for b in range(39):
    i += 1  #pythonはインクリメントがない
    im2 = cv2.imread('/content/drive/MyDrive/working_dir2/keepimg/test' + str(i) + '.jpg')
    im1 = cv2.hconcat([im1, im2]) #横に結合
  
  if a == 0:
    im3 = im1
  else:
    im3 = cv2.vconcat([im3, im1]) #縦に結合
  
  i += 1

#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

#img = mpimg.imread("/content/drive/MyDrive/PBL_data/JPG/output1/chunk_0.jpg")
#imgplot = plt.imshow(im3)
cv2.imwrite('/content/drive/MyDrive/working_dir2/answer.jpg', im3)
    

True